In [1]:
pip install tf-agents

     |████████████████████████████████| 1.2MB 3.9MB/s 
     |████████████████████████████████| 4.8MB 24.2MB/s 
  Found existing installation: tensorflow-probability 0.12.1
    Uninstalling tensorflow-probability-0.12.1:
      Successfully uninstalled tensorflow-probability-0.12.1


In [2]:
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from collections import deque
from collections import defaultdict
import pandas as pd
import numpy as np
import random
import keras
from tf_agents.networks import q_rnn_network
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense
from keras.optimizers import Adam
from tf_agents.environments import py_environment
from tf_agents.environments import tf_environment
from tf_agents.environments import tf_py_environment
from tf_agents.environments import utils
from tf_agents.specs import array_spec
from tf_agents.environments import wrappers
from tf_agents.environments import suite_gym
from tf_agents.trajectories import time_step as ts
import tensorflow as tf
from tf_agents.policies import random_tf_policy
from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.trajectories import trajectory
from tf_agents.utils import common
import os
import tempfile
from tf_agents.policies import policy_saver

from tf_agents.agents.dqn import dqn_agent
from tf_agents.networks import q_network
from tf_agents.utils import common
import sys
import matplotlib.pyplot as plt
from tf_agents.drivers import dynamic_episode_driver

In [24]:
### reading data from csv and saving to dicts


days_prices = defaultdict(dict)
counter = 0
path = "/content/drive/MyDrive/Colab Notebooks/dane_bull"
for files in os.listdir(path):
    counter+=1
    np_dict = defaultdict(dict)
    prices = pd.read_csv(f"/content/drive/MyDrive/Colab Notebooks/dane_bull/{files}", index_col = 0)
    prices = prices.drop("time", axis=1)
    l = len(prices.columns)
    for k in range(0,l):
        prices = pd.read_csv(f"/content/drive/MyDrive/Colab Notebooks/dane_bull/{files}", index_col = 0)
        prices = prices.drop("time", axis=1)
        prices = prices.iloc[:, k:k+1]
        for i in prices.columns:
            prices[f"mov_5{i}"] = prices[f"{i}"].ewm(span=10, adjust=False).mean()
            prices[f"mov_20{i}"] = prices[f"{i}"].rolling(window=60).mean()
            prices[f"mov_60{i}"] = prices[f"{i}"].rolling(window=100).mean()
            prices = prices.dropna()
            prices["5-20"] = np.round((prices[f"mov_5{i}"] - prices[f"mov_20{i}"]), 4)
            scaler_5 = preprocessing.StandardScaler().fit(prices["5-20"].values.reshape(-1,1))
            prices["5-20"] = scaler_5.transform(prices["5-20"].values.reshape(-1,1))
            prices["5-60"] = np.round((prices[f"mov_5{i}"] - prices[f"mov_60{i}"]), 4)
            scaler_5 = preprocessing.StandardScaler().fit(prices["5-60"].values.reshape(-1,1))
            prices["5-60"] = scaler_5.transform(prices["5-60"].values.reshape(-1,1))
            prices["20-60"] = np.round((prices[f"mov_20{i}"] - prices[f"mov_60{i}"]), 4)
            scaler_5 = preprocessing.StandardScaler().fit(prices["20-60"].values.reshape(-1,1))
            prices["20-60"] = scaler_5.transform(prices["20-60"].values.reshape(-1,1))
            if round(prices.iloc[:,0].std(), 7) == 0:
              continue
            np_dict[f"np_diff_{i}"] = np.array([prices["5-20"].values.reshape(-1,1), prices["5-60"].values.reshape(-1,1), prices["20-60"].values.reshape(-1,1), prices[f"{i}"].values.reshape(-1,1) ])
    days_prices[f"np_dict_{counter}"] = np_dict

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

In [25]:
### defining max length of training steps


def max_steps_lenght(days_prices):
    max_steps = 0
    for i in days_prices.keys():
        crypto_dict = days_prices[i].keys()
        number_of_crypto = len(crypto_dict)
        first_dict = list(crypto_dict)[0]
        lenght = len(days_prices[i]["np_diff_XLMUSDT"][0])-60
        lenght = lenght * (number_of_crypto) # timestamp
        max_steps += lenght
    return max_steps
max_steps_lenght(days_prices)

1810842

In [26]:
### creating environment


class CustomEnv(py_environment.PyEnvironment):

    def __init__(self, days_price, initial_balance, lookback_window_size):

        self.lookback_window_size = lookback_window_size

        self._action_spec = array_spec.BoundedArraySpec(
            shape=(), dtype=np.int32, minimum=0, maximum=2, name='action')

        self._observation_spec = array_spec.BoundedArraySpec(
            shape=(self.lookback_window_size * 3 + 1,), dtype=np.float32, name='observation')

        self.days_price = days_price

        self.days_count = 1 

        self.price_dict = days_price[f"np_dict_{self.days_count}"]

        self.dict_names = list(self.price_dict.keys())

        self.dict_count = -1

        self.initial_balance = initial_balance

        self.last_step = len(self.price_dict[f"{self.dict_names[self.dict_count]}"][0]) - self.lookback_window_size

        self.episode_ended = True

        self.active_position = 0

        self.action = 1

        self.fees = 0.001

        self.spread = 0.001

        self.money = initial_balance

        self.current_step = 0

        self.cum_reward = 0

        self.crypto_held = 0

        self.previous_money = initial_balance

        self._state = False
        
        self.price_bought = 0

    def action_spec(self):
        return self._action_spec

    def observation_spec(self):
        return self._observation_spec

    def _reset(self):
        self.episode_ended = False
        self.current_step = 0
        self.price_bought = 0
        self.money = self.initial_balance
        self.active_position = 0
        self.action = 0
        self.crypto_held = 0
        self.dict_count += 1
        if self.dict_count == len(self.dict_names):
            self.days_count += 1
            self.dict_count = 0
            self.price_dict = self.days_price[f"np_dict_{self.days_count}"]
            self.dict_names = list(self.price_dict.keys())
            self.last_step = len(self.price_dict[f"{self.dict_names[self.dict_count]}"][0]) - self.lookback_window_size
        self.mov_price = self.price_dict[f"{self.dict_names[self.dict_count]}"]
        self.previous_money = self.initial_balance
        self.cum_reward = 0  ###Only now
        self.pnl = 0

        self.last = self.current_step + self.lookback_window_size

        self._state = np.array(
            [self.mov_price[0, self.current_step:self.last], self.mov_price[1, self.current_step:self.last],
             self.mov_price[2, self.current_step:self.last]])

        self._state = np.append(self._state, self.active_position)

        return ts.restart(np.array(self._state, dtype=np.float32))

    def open_position(self, action, current_price):
        reward = 0
        if (self.active_position == 0):
            if action == 2:
                self.active_position = 1
                if self.money <= 0.8 * self.initial_balance:
                    self.episode_ended = True
                    reward = -200
                    print('ERROR NO MONEY!')
                else:
                    self.money = self.money - (self.money * self.spread) - (self.money * self.fees)
                    self.crypto_held = (self.money / current_price)
                    reward = self.money - self.previous_money
            
        elif (self.active_position == 1):
            if action == 0:
                self.active_position = 0
                self.money = self.crypto_held * current_price - (self.money * self.spread) - (self.money * self.fees)
                reward = self.money - self.previous_money
                self.crypto_held = 0
            else:
                self.money = self.crypto_held * current_price
                reward = self.money - self.previous_money
        else:
            reward = self.money - self.previous_money

        return reward  # think about money balance

    def next_observation(self):

        self.last = self.current_step + self.lookback_window_size
        previous_price = self.mov_price[3, self.lookback_window_size + self.current_step - 1]
        current_price = self.mov_price[3, self.lookback_window_size + self.current_step]
        obs = np.array([self.mov_price[0, self.current_step:self.last], self.mov_price[1, self.current_step:self.last],
                        self.mov_price[2, self.current_step:self.last]])
        self.current_step += 1
        return obs, current_price, previous_price 

    def _step(self, action):
        reward = 0
        if self.episode_ended == True:
            return self._reset()
        else :
            obs, current_price, previous_price = self.next_observation()
            self._state = obs

        if (action == 0):
          if self.active_position == 0:
              reward = -3
          elif self.active_position == 1:
              reward = self.open_position(action, current_price)
        elif (action == 2):
          if self.active_position == 1:
            reward = self.open_position(action, current_price) - 3
          else: 
            reward = self.open_position(action, current_price)
        elif (action == 1):
            if self.active_position == 1:
                self.money = self.crypto_held * current_price
                reward = self.money - self.previous_money 
            elif self.active_position == 0:
              reward = -1


        if self.last == self.last_step: ##################################################################check should be if last = last.step -1
            self.episode_ended = True

        if self.episode_ended == True:
            self.money = self.crypto_held * current_price - (self.money * self.spread) - (self.money * self.fees)
            self.active_position = 0
            reward = self.money - self.initial_balance

        pnl = (self.money - self.previous_money)
        self.previous_money = self.money
        self.pnl += pnl
        info = {}
        self._state = np.append(self._state, self.active_position)
        # return self._state, reward, done, info
        if self.episode_ended == True:
            return ts.termination(np.array(self._state, dtype=np.float32), float(reward))
        else:
            return ts.transition(
                np.array(self._state, dtype=np.float32), reward=float(reward), discount=1.05)

    def render(self):
        print(f'Step: {self.current_step}, Money: {self.money}, PnL{self.pnl}, Cum reward{self.cum_reward}')
    #####SKALIBROWAĆ REWARD




In [27]:
#### creating model and agent


class Agent_net():

    def __init__(self, env):
        # defining hyperparameters
        self.env = env

        self.train_env = tf_py_environment.TFPyEnvironment(self.env)
        self.eval_env = tf_py_environment.TFPyEnvironment(self.env)
        # network configuration
        self.input_fc_layer_params = (181,)
        self.lstm_size = (60,)
        self.output_fc_layer_params = (60,)
        self.learning_rate = 0.005
        self.optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=self.learning_rate)
        self.td_errors_loss_fn = common.element_wise_squared_loss
        self.train_step_counter = tf.compat.v1.train.get_or_create_global_step() #tf.Variable(0)  
        self.replay_buffer_max_length = 1000000
        self.batch_size = 64
        self.epsilon = 0.2

    def qrnn(self):
        # create a q_RNNnet
        self.q_net = q_rnn_network.QRnnNetwork(
            self.train_env.observation_spec(),
            self.train_env.action_spec(),
            input_fc_layer_params=self.input_fc_layer_params,
            lstm_size=self.lstm_size,
            output_fc_layer_params=self.output_fc_layer_params)

    def agent_create(self):
        self.qrnn()
        self.target = self.qrnn()
        self.agent = dqn_agent.DqnAgent(
            self.train_env.time_step_spec(),
            self.train_env.action_spec(),
            q_network=self.q_net,
            target_q_network = self.target,
            target_update_tau = 0.1,
            target_update_period = 10,
            epsilon_greedy = self.epsilon,
            optimizer=self.optimizer,
            td_errors_loss_fn=self.td_errors_loss_fn,
            train_step_counter=self.train_step_counter)

        self.agent.initialize() # just to start agent
        self.eval_policy = self.agent.policy # return the policy
        self.random_policy = random_tf_policy.RandomTFPolicy(self.train_env.time_step_spec(),
                                                             self.train_env.action_spec())
        return (self.agent, self.train_env, self.eval_env)



In [28]:
def r_buffer(agent, train_env, replay_buffer_max_length):
    replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
        data_spec=agent.collect_data_spec, # getting data scpecification 
        batch_size=train_env.batch_size, # how many experience there are in a batch
        max_length=replay_buffer_max_length) # maximum capacity of buffer is max_lenght * batch size (stored in a tuple)
    return replay_buffer

In [29]:
def driver_collect(replay_buffer, train_env, agent):
  replay_buffer.clear()
  observers = [replay_buffer.add_batch]       
  driver = dynamic_episode_driver.DynamicEpisodeDriver(
            train_env, agent.collect_policy, observers, num_episodes=1)
  driver.run()
  return replay_buffer
  

In [30]:
def iter_data(replay_buffer, train_env):
  AUTOTUNE = tf.data.experimental.AUTOTUNE
  ns = replay_buffer.num_frames()
  ns = ns.numpy()
  dataset = replay_buffer.as_dataset(
      num_parallel_calls=AUTOTUNE, #how many process at time, if none sequential
      sample_batch_size=train_env.batch_size, #how many batches to get
      num_steps=ns).prefetch(AUTOTUNE) #how many steps in a batch  
  iterator = iter(dataset)
  return (iterator)

In [31]:
### computing reward returned in order to work collect_step() must be first



def compute_avg_return(environment, policy, num_eval_episodes):

    total_return = 0.0
    for _ in range(num_eval_episodes): # number o evaluation episodes

        time_step = environment.reset() # getting first time_step
        episode_return = 0
        policy_state = policy.get_initial_state(batch_size=env.batch_size)
        avg_return = 0
        

        while not time_step.is_last():
            action_step = policy.action(time_step, policy_state) # action taken for given timestep
            time_step = environment.step(action_step.action) # assign next time_step 
            episode_return += time_step.reward # reward taken from next time_step ?
           
        total_return+= episode_return
        print(episode_return)
    avg_return = total_return  / num_eval_episodes
    return avg_return

In [32]:
# agent=None
# env = None
# tf.compat.v1.reset_default_graph()
# env = CustomEnv(days_prices, 1000, 60)
# na = Agent_net(env)
# agent, train_env, eval_env = na.agent_create()
# replay_buffer = r_buffer(agent, train_env, 500000)
# count = 0
# for _ in range(119):  #######docelowo 120*len(days_prices.keys())
#       count+=1
#       replay_buffer = driver_collect(replay_buffer, train_env, agent)
#       iterator = iter_data(replay_buffer, train_env)
#       print(iterator.get_next())

In [33]:
### training


def training(agent, train_env, replay_buffer, eval_env, iterator):
    num_iterations = 1000000  
    collect_steps_per_iteration = 10 
    log_interval = 30
    eval_interval = 100
    num_eval_episodes = 3
    index_out = False
    n_episodes = 120 * len(days_prices.keys())
    try:
        % % time # used for prolonged training
    except:
        pass

    # (Optional) Optimize by wrapping some of the code in a graph using TF function.
    agent.train = common.function(agent.train)

    # Evaluate the agent's policy once before training.
    avg_return = compute_avg_return(eval_env, agent.policy, num_eval_episodes) #use created function
    returns = [avg_return]

    for _ in range(n_episodes):
          try:
            replay_buffer = driver_collect(replay_buffer, train_env, agent)
            iterator = iter_data(replay_buffer, train_env)
            

            # Sample a batch of data from the buffer and update the agent's network.
            experience, unused_info = iterator.get_next() 
            train_loss = agent.train(experience).loss
          except IndexError:
            index_out = True

          step = agent.train_step_counter.numpy()

          if step % log_interval == 0:
              print('step = {0}: loss = {1}'.format(step, train_loss))
              
          if step % eval_interval == 0:
              try:
                  avg_return = compute_avg_return(eval_env, agent.policy, num_eval_episodes)
                  print('step = {0}: Average Return = {1}'.format(step, avg_return))
                  returns.append(avg_return)
              except Exception:
                  print("index error")
                  index_out = True
          if index_out == True:
            break

    return agent.train_step_counter, agent, replay_buffer

In [ ]:
### calling env, agent creation, training etc
agent=None
env = None
tf.compat.v1.reset_default_graph()
env = CustomEnv(days_prices, 1000, 60)
#utils.validate_py_environment(env, episodes=1)
na = Agent_net(env)
agent, train_env, eval_env = na.agent_create()
replay_buffer = r_buffer(agent, train_env, 50000)
iterator = iter_data(replay_buffer, train_env)
save_train_step_counter, save_agent, save_replay_buffer = training(agent, train_env, replay_buffer, eval_env, iterator)

chkpdir = "/content/drive/MyDrive/Colab Notebooks/policies"
policy_dir = os.path.join(chkpdir, 'policy')
checkpoint_dir = os.path.join(chkpdir, 'checkpoint')
train_checkpointer = common.Checkpointer(
    ckpt_dir=checkpoint_dir,
    max_to_keep=1,
    model = na.q_net,
    agent=save_agent,
    policy=save_agent.policy,
    replay_buffer=save_replay_buffer,
    global_step=save_agent.train_step_counter
)

tf_policy_saver = policy_saver.PolicySaver(save_agent.policy)
checkpoint = train_checkpointer.save(save_agent.train_step_counter)
#train_checkpointer.initialize_or_restore()
#global_step = tf.compat.v1.train.get_global_step()
tf_policy_saver.save(policy_dir)

tf.Tensor([-2258.629], shape=(1,), dtype=float32)
tf.Tensor([-2240.5103], shape=(1,), dtype=float32)
tf.Tensor([-2196.9285], shape=(1,), dtype=float32)
step = 30: loss = 1002062.3125
step = 60: loss = 998974.25
step = 90: loss = 2814.17236328125
tf.Tensor([-23.262459], shape=(1,), dtype=float32)
tf.Tensor([-181.70383], shape=(1,), dtype=float32)
tf.Tensor([-140.60687], shape=(1,), dtype=float32)
step = 100: Average Return = [-115.19105]
step = 120: loss = 8380.2138671875
step = 150: loss = 20773.23828125
step = 180: loss = 20365.13671875
ERROR NO MONEY!
tf.Tensor([-1009.04846], shape=(1,), dtype=float32)
tf.Tensor([-1009.7992], shape=(1,), dtype=float32)
tf.Tensor([-388.26544], shape=(1,), dtype=float32)
step = 200: Average Return = [-802.37103]
ERROR NO MONEY!
ERROR NO MONEY!
step = 210: loss = 999594.75
ERROR NO MONEY!
ERROR NO MONEY!
ERROR NO MONEY!
ERROR NO MONEY!
step = 240: loss = 905.3056640625
step = 270: loss = 4956.59326171875
step = 300: loss = 998563.375
tf.Tensor([-1470.],

In [14]:
chkpdir = "/content/drive/MyDrive/Colab Notebooks/policies"
policy_dir = os.path.join(chkpdir, 'policy')
checkpoint_dir = os.path.join(chkpdir, 'checkpoint')
train_checkpointer = common.Checkpointer(
    ckpt_dir=checkpoint_dir,
    max_to_keep=1,
    model = na.q_net,
    agent=save_agent,
    policy=save_agent.policy,
    global_step=save_agent.train_step_counter
)

tf_policy_saver = policy_saver.PolicySaver(save_agent.policy)
checkpoint = train_checkpointer.save(save_agent.train_step_counter)
#train_checkpointer.initialize_or_restore()
#global_step = tf.compat.v1.train.get_global_step()
tf_policy_saver.save(policy_dir)


FOR DEVS: If you are overwriting _tracking_metadata in your class, this property has been used to save metadata in the SavedModel. The metadta field will be deprecated soon, so please move the metadata to a different file.



FOR DEVS: If you are overwriting _tracking_metadata in your class, this property has been used to save metadata in the SavedModel. The metadta field will be deprecated soon, so please move the metadata to a different file.


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/policies/policy/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/policies/policy/assets


In [ ]:
policy = save_agent.policy

In [ ]:
for d in days_prices.keys():
  PnL = 0
  for k in np_dict.keys():
    balance = 1000
    spread = 0.001
    fees = 0.001
    crypto_held = 0
    active_position = 0
    policy_state = policy.get_initial_state(1)
    a_count = 0
    pos = 0
    for i in range(0, 100):
        price_mov_data = np_dict[f"{k}"][0:3, i:i + 60].reshape(180, )
        price = np_dict[f"{k}"][3, i]
        
        

        price_mov_data = np.append([price_mov_data], [active_position])

        ts_obs = tf.convert_to_tensor(price_mov_data)

        step_type = tf.convert_to_tensor(
            [0], dtype=tf.int32, name='time_step')
        reward = tf.convert_to_tensor(
            [0], dtype=tf.float32, name='reward')
        discount = tf.convert_to_tensor(
            [0], dtype=tf.float32, name='discount')
        observations = tf.convert_to_tensor(
            [ts_obs], dtype=tf.float32, name='observations') 
        timestep = ts.TimeStep(step_type, reward, discount, observations)

        # def runPolicy(timestep, _policy, _policy_state, balance, fees, crypto_held, active_position, price):

        a = policy.action(timestep, policy_state)
        policy_state = a.state
        a = a.action.numpy()
        #a = random.randint(0,2)
        #a=1
        #print(a)
      
        if active_position == 0:
            if int(a) == 1:
                active_position = 1
                if balance <= 1:
                    print('ERROR NO MONEY!')
                else:
                    balance = balance - (balance * spread) - (balance * fees)
                    crypto_held = (balance / price)
        elif active_position == 1:
            if int(a) == 0:
                active_position = 0
                balance = crypto_held * price - (balance * spread) - (balance * fees)
                crypto_held = 0
            else:
                balance = crypto_held * price
        else:
            balance = crypto_held * price
        
        if int(a) == 1:
          a_count+= 1

        pnl = 1000 - balance
      
        
        
    
    PnL += -pnl
    print(-pnl, a_count)

  print(PnL)
# runPolicy(timestep, policy, policy_state, balance, fees, crypto_held, active_position, price)

0 0
[1.13295922] 25
[9.67759023] 30
[12.85416667] 41
0 0
[22.36008345] 25
[13.48324459] 26
0 0
[69.3095996] 53
0 0
[-3.24103627] 17
[-1.09895269] 43
[6.99639423] 20
[11.94083813] 22
[-3.7204693] 24
[53.2515708] 51
[40.2331012] 39
[-5.12363067] 18
[3.6572367] 21
[14.61262507] 22
0 0
[-3.61265005] 21
0 0
[0.37069989] 21
[2.87870455] 19
[-2.24818462] 24
[-0.96201689] 20
[-3.81950775] 18
[2.24476662] 24
[-6.06194812] 21
[5.51113138] 24
[-5.57108779] 25
[12.1502937] 25
[-12.53115613] 19
0 0
[-9.33978836] 100
[2.93652102] 22
[-4.47816464] 26
0 0
[-13.50497288] 24
[5.83498053] 21
[4.95999161] 24
[14.77532425] 22
[-5.23594945] 18
[6.42215894] 22
[7.72849626] 22
[6.87111111] 23
[-2.4932536] 24
[0.19555121] 18
[9.05930638] 24
[-3.23082833] 19
[38.86058429] 36
[-5.34395644] 26
0 0
[10.59867342] 25
[14.89550496] 80
[2.81078504] 21
[0.37392959] 21
[-3.34798977] 25
[-3.89434254] 18
[-2.9266481] 18
[1.43537982] 20
0 0
[-4.73866021] 40
[-10.95347382] 20
[-3.31801104] 23
[0.63884578] 100
[0.12896512] 7

KeyboardInterrupt: ignored